In [14]:
import os

EMAIL_ADDRESS = "adhammenisii@gmail.com"  
EMAIL_PASSWORD = "wnckrhihhkrcobcg" 

IMAP_SERVER = "imap.gmail.com"
IMAP_PORT = 993
INBOX_FOLDER = "INBOX"

print("Using:", EMAIL_ADDRESS)
print("App Password is set")

Using: adhammenisii@gmail.com
App Password is set


In [15]:
import imaplib, email
from typing import List, Dict, Tuple, Optional
import time

def simple_html_to_text(html_text):
    import re
    if not html_text:
        return ""
    text = re.sub(r'<[^>]+>', ' ', html_text)
    text = re.sub(r'\s+', ' ', text)
    return text.strip()

class ImapClient:
    def __init__(self, server: str, port: int, address: str, password: str):
        self.server = server
        self.port = port
        self.address = address
        self.password = password
        self.conn = None

    def connect(self):
        try:
            print(f"Connecting to {self.server}:{self.port}...")
            self.conn = imaplib.IMAP4_SSL(self.server, self.port)
            print("SSL connection established, logging in...")
            self.conn.login(self.address, self.password)
            print("✅ IMAP connected successfully!")
        except Exception as e:
            print(f"❌ Connection failed: {e}")
            return False
        return True

    def list_folders(self) -> List[str]:
        try:
            typ, data = self.conn.list()
            return [line.decode('utf-8', errors='ignore') for line in (data or [])]
        except:
            return []

    def select(self, folder: str):
        self.conn.select(folder)

    def uid_search(self, criterion: str) -> List[bytes]:
        st, d = self.conn.uid('SEARCH', None, criterion)
        if st != 'OK' or not d or not d[0]:
            return []
        return d[0].split()

    def uid_fetch_rfc822(self, uid: bytes):
        st, data = self.conn.uid('FETCH', uid, '(RFC822)')
        if st != 'OK' or not data:
            return None
        for part in data:
            if isinstance(part, tuple) and len(part) >= 2:
                return part[1]
        return None

    def uid_move(self, uid: bytes, dest_folder: str) -> bool:
        try:
            st, _ = self.conn.uid('COPY', uid, dest_folder)
            if st == 'OK':
                self.conn.uid('STORE', uid, '+FLAGS', r'(\Deleted)')
                self.conn.expunge()
                return True
        except:
            pass
        return False

    def uid_mark_seen(self, uid: bytes) -> bool:
        try:
            st, _ = self.conn.uid('STORE', uid, '+FLAGS', r'(\Seen)')
            return st == 'OK'
        except:
            return False

def extract_text_from_message(msg):
    if not msg:
        return ""
    
    text_plain = ""
    text_html = ""
    
    if msg.is_multipart():
        for part in msg.walk():
            content_type = part.get_content_type()
            content_disposition = str(part.get('Content-Disposition', ''))
            
            if 'attachment' in content_disposition:
                continue
                
            if content_type == 'text/plain' and not text_plain:
                try:
                    payload = part.get_payload(decode=True)
                    if payload:
                        text_plain = payload.decode('utf-8', errors='ignore')
                except:
                    pass
                    
            elif content_type == 'text/html' and not text_html:
                try:
                    payload = part.get_payload(decode=True)
                    if payload:
                        text_html = payload.decode('utf-8', errors='ignore')
                except:
                    pass
    else:
        content_type = msg.get_content_type()
        try:
            payload = msg.get_payload(decode=True)
            if payload:
                if content_type == 'text/plain':
                    text_plain = payload.decode('utf-8', errors='ignore')
                elif content_type == 'text/html':
                    text_html = payload.decode('utf-8', errors='ignore')
        except:
            pass

    if text_plain:
        return text_plain
    elif text_html:
        return simple_html_to_text(text_html)
    else:
        return ""

# الإتصال
print("🔄 Attempting connection...")
client = ImapClient(IMAP_SERVER, IMAP_PORT, EMAIL_ADDRESS, EMAIL_PASSWORD)
success = client.connect()

if success:
    print("🎉 Ready to proceed to next cell!")
else:
    print("💔 Check your email and password")

🔄 Attempting connection...
Connecting to imap.gmail.com:993...
SSL connection established, logging in...
✅ IMAP connected successfully!
🎉 Ready to proceed to next cell!


In [ ]:
import re, string, io, zipfile, requests, pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report

def simple_preprocess_text(t: str) -> str:
    if not isinstance(t, str):
        return ""
    t = t.lower()
    t = re.sub(r"\S+@\S+", "emailaddr", t)
    t = re.sub(r"http[s]?://\S+", "httpaddr", t)
    t = re.sub(r"\d+", "number", t)
    t = t.translate(str.maketrans("", "", string.punctuation))
    
    words = t.split()
    simple_stop_words = {'the', 'a', 'an', 'and', 'or', 'but', 'in', 'on', 'at', 'to', 'for', 'of', 'with', 'by'}
    words = [w for w in words if w not in simple_stop_words and len(w) > 2]
    
    return " ".join(words)

print("📥 Downloading data...")
url = "https://archive.ics.uci.edu/ml/machine-learning-databases/00228/smsspamcollection.zip"
z = zipfile.ZipFile(io.BytesIO(requests.get(url, timeout=30).content))
df = pd.read_csv(io.BytesIO(z.read("SMSSpamCollection")), sep="\t", names=["label","text"])
df["y"] = (df["label"] == "spam").astype(int)

print("🔄 Preprocessing texts...")
texts = [simple_preprocess_text(x) for x in df["text"].tolist()]
Xtr, Xte, ytr, yte = train_test_split(texts, df["y"].values, test_size=0.2, random_state=42, stratify=df["y"].values)

print("🤖 Training model...")
clf = Pipeline([
    ("vec", TfidfVectorizer(max_features=10000, ngram_range=(1,2), min_df=2)),
    ("lr", LogisticRegression(max_iter=500, class_weight="balanced"))
])
clf.fit(Xtr, ytr)

print("📊 Evaluating...")
pred = clf.predict(Xte)
print("✅ Accuracy:", f"{accuracy_score(yte, pred):.3f}")
print(classification_report(yte, pred, digits=3))
print("🎉 Model ready for spam detection!")

📥 Downloading data...
🔄 Preprocessing texts...
🤖 Training model...
📊 Evaluating...
✅ Accuracy: 0.972
              precision    recall  f1-score   support

           0      0.993     0.975     0.984       966
           1      0.855     0.953     0.902       149

    accuracy                          0.972      1115
   macro avg      0.924     0.964     0.943      1115
weighted avg      0.974     0.972     0.973      1115

🎉 Model ready for spam detection!


In [17]:

SPAM_FOLDER = "[Gmail]/Spam"  

try:
    folders = client.list_folders()
    spam_like = []
    for s in folders:
        if any(x in s for x in ["Spam", "Junk", "\\Junk", "\\Spam", "junk", "spam"]):
            spam_like.append(s)
    def _extract_name(s: str) -> str:
        import re
        m = list(re.finditer(r'"([^"]+)"', s))
        if m:
            return m[-1].group(1)
        parts = s.split(' ")')
        return parts[-1].strip('"') if parts else s
    if spam_like:
        SPAM_FOLDER = _extract_name(spam_like[0])
    print("Using SPAM_FOLDER =", SPAM_FOLDER)
except Exception as e:
    print("Folder detection failed:", e)

try:
    client.select(INBOX_FOLDER)
    st_inbox = client.conn.status(INBOX_FOLDER, '(MESSAGES)')
    client.select(SPAM_FOLDER)
    st_spam = client.conn.status(SPAM_FOLDER, '(MESSAGES)')
    print("Status INBOX:", st_inbox)
    print("Status SPAM:", st_spam)
except Exception as e:
    print("Diagnostics failed:", e)


Using SPAM_FOLDER = [Gmail]/Spam
Status INBOX: ('OK', [b'"INBOX" (MESSAGES 591)'])
Status SPAM: ('OK', [b'"[Gmail]/Spam" (MESSAGES 5)'])


In [18]:
from numpy import max as npmax

CONFIDENCE_THRESHOLD = 0.7

def predict_spam(subject: str, body: str):
    # استخدم simple_preprocess_text بدل preprocess_text
    text = simple_preprocess_text(f"{subject} {body}")
    proba = clf.predict_proba([text])[0]
    pred = clf.predict([text])[0]
    return bool(pred), float(npmax(proba))

def get_unseen_uids(limit: int = 100):
    client.select(INBOX_FOLDER)
    uids = client.uid_search('UNSEEN')
    return uids[-limit:]

def fetch_email_by_uid(uid):
    raw = client.uid_fetch_rfc822(uid)
    if not raw:
        return {"uid": uid.decode() if isinstance(uid, (bytes, bytearray)) else str(uid), "subject": "", "body": ""}
    msgobj = email.message_from_bytes(raw)
    body = extract_text_from_message(msgobj)
    return {
        "uid": uid.decode() if isinstance(uid, (bytes, bytearray)) else str(uid),
        "subject": msgobj.get('Subject','') or '',
        "sender": msgobj.get('From','') or '',
        "body": body,
    }

def process_once(max_messages: int = 100, dry_run: bool = True):
    uids = get_unseen_uids(limit=max_messages)
    print(f"📧 Considering {len(uids)} unseen messages")
    moved = 0
    read = 0
    for uid in uids:
        item = fetch_email_by_uid(uid)
        is_spam, conf = predict_spam(item['subject'], item['body'])
        if is_spam and conf >= CONFIDENCE_THRESHOLD:
            action = f"🚫 SPAM({conf:.2f}) -> move"
            if not dry_run and client.uid_move(uid, SPAM_FOLDER):
                moved += 1
        else:
            action = f"✅ HAM({conf:.2f}) -> mark read"
            if not dry_run and client.uid_mark_seen(uid):
                read += 1
        subj = (item['subject'] or '').replace('\n',' ')[:80]
        print(f"{action}: {subj}")
    print(f"🎉 Done. Moved: {moved}, Read: {read}, dry_run={dry_run}")

print("✅ All functions are ready! You can now run process_once()")

✅ All functions are ready! You can now run process_once()


In [19]:

process_once(max_messages=50, dry_run=True)


📧 Considering 50 unseen messages
🚫 SPAM(0.88) -> move: Adham, win work with these top AI skills
🚫 SPAM(0.96) -> move: Your ride awaits
✅ HAM(0.67) -> mark read: Important updates in Gemini Apps
🚫 SPAM(0.97) -> move: =?UTF-8?Q?Women=E2=80=99s_fashion:_discounts_on_the_way?=
🚫 SPAM(0.90) -> move: ae173712.user,Our hottest items, now over 50% off
🚫 SPAM(0.86) -> move: =?UTF-8?B?TmV3IGFubm91bmNlbWVudDogItin2YTYp9mF2KrYrdin2YYg2YTZhNmG2KfYsyDYp9mE?=
🚫 SPAM(0.85) -> move: =?UTF-8?B?TmV3IGFubm91bmNlbWVudDogItin2K7YqtmK2KfYsSDZhdmI2KfYudmK2K8g2KfZhNmF?=
🚫 SPAM(0.96) -> move: ae173712.user,Our top-rated items out there!
  Console with > move: It's selling real quick: Kinhank Super Console X5 Pro Retro Game
🚫 SPAM(0.86) -> move: =?UTF-8?B?TmV3IGFubm91bmNlbWVudDogItin2K7YsSDZhdi52KfYryDZhNiq2LPZhNmK2YUg2KfZig
🚫 SPAM(0.95) -> move: Welcome to TikTok
🚫 SPAM(0.88) -> move: 804185 is your verification code
🚫 SPAM(0.88) -> move: 804185 is your verification code
🚫 SPAM(0.89) -> move: People are reviewi

In [ ]:
import threading
import time

_schedule_stop = threading.Event()

def _job():
    try:
        print("🔄 Running scheduled spam check...")
        process_once(max_messages=100, dry_run=False)
        print("✅ Scheduled check completed")
    except Exception as e:
        print("❌ Scheduled run failed:", e)

def start_scheduler():
    def loop():
        while not _schedule_stop.is_set():
            # تشغيل كل 6 ساعات (21600 ثانية)
            _job()
            # إنتظار 6 ساعات
            for _ in range(21600):
                if _schedule_stop.is_set():
                    break
                time.sleep(1)
    
    t = threading.Thread(target=loop, daemon=True)
    t.start()
    print("✅ Scheduler started: will run every 6 hours")

def stop_scheduler():
    _schedule_stop.set()
    print("🛑 Scheduler stopped")

# بدء التشغيل
start_scheduler()
print("🎉 Spam detector is now running automatically!")
print("💡 To stop: run stop_scheduler() in a new cell")

📧 Considering 100 unseen messages
🚫 SPAM(0.95) -> move: 6 shoppers are snagging these
🚫 SPAM(0.96) -> move: ae173712.user,Our hottest items, now over 50% off
🚫 SPAM(0.88) -> move: You're going places
✅ HAM(0.50) -> mark read: Good news alert!
🚫 SPAM(0.86) -> move: 2 ordered this home item
🚫 SPAM(0.97) -> move: 20% Off This Fathers Day Celebration
  Options from 48 -ve: Recommended for home: 2024 New SM10 Sand Club: Multiple Loft
✅ HAM(0.50) -> mark read: Good news alert!
🚫 SPAM(0.97) -> move: 50% Off Lifetime Deal on All Plans
🚫 SPAM(0.97) -> move: Weekly Debutify Insight!
🚫 SPAM(0.97) -> move: It's selling real quick: EOENKK Block Chain Miner & Accessories
 	=?utf-8?B?cy4> move: =?utf-8?B?8J+agCAyMiBwYWdlcy4gNiBtb250aHMuIFplcm8gZGV2ZWxvcGVy?=
🚫 SPAM(0.86) -> move: PSA: weekly bestsellers are here
🚫 SPAM(0.97) -> move: ae173712.user,Our hottest items, now over 50% off
🚫 SPAM(0.87) -> move: 65 ordered this home item
  =?UTF-8?Q?_=E2=80=93_NeUTF-8?Q?=F0=9F=9A=80_Debutify_AI_Theme_8.3_is_